In [2]:
from CANdb_functions import *
import numpy as np
import re


In [6]:
messages_and_signals = pd.read_csv("CSVs/RAW_MESSAGES.csv")
print(messages_and_signals.columns)


Index(['MSG ID', 'Message Name', 'Length', 'Byte', 'Bit Start', 'Data Name',
       'Description', 'Data Type', 'Min', 'Max', 'Scaled Min', 'Scaled Max',
       'Map Equation', 'Units', 'For Indexing Purposes'],
      dtype='object')


In [ ]:
messages = messages_and_signals[["MSG ID", "Message Name", "Length"]].dropna(subset=["Message Name"])
print(messages_and_signals.head(20))
print(messages.info())


# get unique messages, signals later


   MSG ID  Message Name  Length  Byte Bit Start         Data Name  \
0   0x000     Debug 2.0     8.0   NaN       NaN               NaN   
1     NaN           NaN     NaN   0-7         0             Debug   
2   0x001      Debug FD    64.0   NaN       NaN               NaN   
3     NaN           NaN     NaN  0-63         0             Debug   
4   0x002          Ping     4.0   NaN       NaN               NaN   
5     NaN           NaN     NaN   0-3         0         Timestamp   
6   0x003  ECU Status 1     8.0   NaN       NaN               NaN   
7     NaN           NaN     NaN     0         0         ECU State   
8     NaN           NaN     NaN   1-3         8  ACU Node Status    
9     NaN           NaN     NaN   NaN         9   GR Inv 1 Status   
10    NaN           NaN     NaN   NaN        10   GR Inv 2 Status   
11    NaN           NaN     NaN   NaN        11   GR Inv 3 Status   
12    NaN           NaN     NaN   NaN        12   GR Inv 4 Status   
13    NaN           NaN     NaN   

In [ ]:
# funny solution because message sheets format is dog

CANdb = cantools.database.load_file(base_dbc)

nodes = [node.name for node in CANdb.nodes]

# Function to find a sender node by matching first 3 letters
def find_sender(message_name):
    prefix = message_name[:3].replace(" ", "_").lower()  
    matches = [n for n in nodes if prefix in n.lower()]   
    return matches[0] if matches else "ECU" 


messages["Sender"] = messages["Message Name"].apply(find_sender)

print(messages.head())
messages.to_csv("CSVs/MESSAGES.csv", index=False)


    MSG ID                       Message Name  Length    Sender
0    0x000                          Debug 2.0     8.0  Debugger
2    0x001                           Debug FD    64.0  Debugger
4    0x002                               Ping     4.0       ECU
6    0x003                       ECU Status 1     8.0       ECU
29   0x004                       ECU Status 2     8.0       ECU
34   0x005                       ECU Status 3     4.0       ECU
37   0x006                         ECU config     0.0       ECU
39   0x007                       ACU Status 1     8.0       ECU
45   0x008                       ACU Status 2     8.0       ECU
63   0x009                       ACU Status 3     8.0       ECU
68   0x00A                      ACU Precharge     1.0       ECU
70   0x00B       ACU Config Charge Parameters     4.0       ECU
73   0x00C  ACU Config Operational Parameters     2.0       ECU
76   0x00D                    ACU Cell Data 1    64.0       ECU
141  0x00E                    ACU Cell D

In [9]:
signals = messages_and_signals[["Message Name", "Length", "Bit Start", "Data Name", "Description", 
                                "Min", "Max", "Map Equation", "Units"]]

signals["Message Name"] = signals["Message Name"].fillna(method = "ffill")
signals["Length"] = signals["Length"].fillna(method="ffill")
signals = signals.dropna(subset=["Data Name"])

signals = signals.rename(columns = {"Length" : "Message Length",
                                    "Description" : "Comment",
                                    "Data Name" : "Signal Name",
                                    "Map Equation" : "Factor",
                                    "Units" : "Unit"})

signals[["Signal Length", "Byte Order", "Signed"]] = 0, "big_endian", False

"""
-- Necessary info for signals
Message Name
Message ID
DLC
Sender

^ message info

Signal Name
Start Bit - 
Length (Bits) - need to calculate
Byte Order - all big_endian
Signed - all false change later
Factor - default 1 change later
Offset
Min Value - default nan change later
Max Value - default nan change later
Unit
Multiplexing Mode
Comment
"""

print(signals.head(20))


    Message Name  Message Length Bit Start              Signal Name  \
1      Debug 2.0             8.0         0                    Debug   
3       Debug FD            64.0         0                    Debug   
5           Ping             4.0         0                Timestamp   
7   ECU Status 1             8.0         0                ECU State   
8   ECU Status 1             8.0         8         ACU Node Status    
9   ECU Status 1             8.0         9          GR Inv 1 Status   
10  ECU Status 1             8.0        10          GR Inv 2 Status   
11  ECU Status 1             8.0        11          GR Inv 3 Status   
12  ECU Status 1             8.0        12          GR Inv 4 Status   
13  ECU Status 1             8.0        13         Fan Controller 1   
14  ECU Status 1             8.0        14         Fan Controller 2   
15  ECU Status 1             8.0        15         Fan Controller 3   
16  ECU Status 1             8.0        16         Fan Controller 4   
17  EC

C:\Users\uclam\AppData\Local\Temp\ipykernel_16988\3597818750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals["Message Name"] = signals["Message Name"].fillna(method = "ffill")
C:\Users\uclam\AppData\Local\Temp\ipykernel_16988\3597818750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  signals["Length"] = signals["Length"].fillna(method="ffill")


In [10]:
prev_value = 0
def safe_convert(value):
    global prev_value
    try:
        prev_value = int(value)
    except ValueError:
        prev_value += 1
    return prev_value

signals["Bit Start"] = signals["Bit Start"].apply(safe_convert)

In [11]:
def safe_convert_to_float(value):
    try:
        return (float(value))  # Convert to int, allowing float conversion first
    except ValueError:
        return np.nan  # Replace with NaN if conversion fails

signals["Min"] = signals["Min"].apply(safe_convert_to_float)
signals["Max"] = signals["Max"].apply(safe_convert_to_float)

signals["Factor"] = signals["Factor"].astype(str).apply(lambda x: re.sub(r'[a-zA-Z]', '', x))
signals["Factor"] = signals["Factor"].apply(safe_convert_to_float)

signals.index = range(len(signals))

print(signals.head(10))

   Message Name  Message Length  Bit Start       Signal Name  \
0     Debug 2.0             8.0          0             Debug   
1      Debug FD            64.0          0             Debug   
2          Ping             4.0          0         Timestamp   
3  ECU Status 1             8.0          0         ECU State   
4  ECU Status 1             8.0          8  ACU Node Status    
5  ECU Status 1             8.0          9   GR Inv 1 Status   
6  ECU Status 1             8.0         10   GR Inv 2 Status   
7  ECU Status 1             8.0         11   GR Inv 3 Status   
8  ECU Status 1             8.0         12   GR Inv 4 Status   
9  ECU Status 1             8.0         13  Fan Controller 1   

                                             Comment  Min  Max  Factor  Unit  \
0  Essentially a print statement up to 8 bytes lo...  NaN  NaN     NaN   NaN   
1  Essentially a print statement up to 64 bytes l...  NaN  NaN     NaN   NaN   
2                                     Time in millis  0

In [12]:
signals.reset_index(drop=True)

signal_lengths = []

for i in range(len(signals)):
    if i < len(signals) - 1 and signals.loc[i, "Message Name"] == signals.loc[i + 1, "Message Name"]:
        # Calculate length as difference between next and current Bit Start
        length = signals.loc[i + 1, "Bit Start"] - signals.loc[i, "Bit Start"]
    elif (signals.loc[i, "Bit Start"] >= signals.loc[i, "Message Length"] * 8):
        length = 1
        signals.at[i, "Bit Start"] = 0
    else:
        length = int(signals.loc[i, "Message Length"] * 8) - signals.loc[i, "Bit Start"]

    signal_lengths.append(length)

signals["Signal Length"] = signal_lengths
print(signals.head(10))
print(signals.info())
signals.to_csv("CSVs/SIGNALS.csv", index=False)


   Message Name  Message Length  Bit Start       Signal Name  \
0     Debug 2.0             8.0          0             Debug   
1      Debug FD            64.0          0             Debug   
2          Ping             4.0          0         Timestamp   
3  ECU Status 1             8.0          0         ECU State   
4  ECU Status 1             8.0          8  ACU Node Status    
5  ECU Status 1             8.0          9   GR Inv 1 Status   
6  ECU Status 1             8.0         10   GR Inv 2 Status   
7  ECU Status 1             8.0         11   GR Inv 3 Status   
8  ECU Status 1             8.0         12   GR Inv 4 Status   
9  ECU Status 1             8.0         13  Fan Controller 1   

                                             Comment  Min  Max  Factor  Unit  \
0  Essentially a print statement up to 8 bytes lo...  NaN  NaN     NaN   NaN   
1  Essentially a print statement up to 64 bytes l...  NaN  NaN     NaN   NaN   
2                                     Time in millis  0